In [66]:
import pandas as pd
import numpy as np
import pickle 
from datetime import datetime
from fuzzywuzzy import fuzz

with open('riaa_df.pkl', 'rb') as f:
        riaa_df = pickle.load(f)
with open('hot-100_chart_final.pkl', 'rb') as f:
        rec_df= pickle.load(f)     
with open('song_df_aggregate.pkl', 'rb') as f:
        song_df = pickle.load(f)

In [ ]:
riaa_df["award_datetime"] = riaa_df["award_date"].apply(lambda x: datetime.strptime(x.strip(),"%B %d, %Y"))
song_df["spotify_album_release_date_datetime"] = song_df["spotify_album_release_date"].apply(lambda x: datetime.strptime(x.strip(),"%Y-%m-%d") if len(str(x))>8 else x)

In [ ]:
riaa_df["award"] = riaa_df["award"].apply(lambda x: x.strip())
keywords = ["Featuring","Feauring", "featuring", "&",","," X ", " x ","Presents", "From", "Or","And",]
for song in song_df.index:
    date = song_df.loc[song,"spotify_album_release_date_datetime"]
    artist_df = riaa_df[riaa_df["award_datetime"] <= date]
    artist = song_df.loc[song,"artist"]
    last_award_list = list()
    award_num_list = list()
    num_songs_awards = list()
    gold_count = list()
    platinum_count = list()
    
    artist_list = list()
    artist_list.append(artist)
    for keyword in keywords:
        artist_list.extend(artist.split(keyword))
    artist_list = list(set(artist_list))
    
    if len(str(date)) >8:
        for performer in artist_list:
            artist_df1 = artist_df[artist_df["artist"] == performer.strip()].sort_values("award_datetime",ascending=False)
            if len(artist_df1)>0:
                gold_count.append(artist_df1[artist_df1["award"] == "Gold"].count()["award"])
                artist_df1 = artist_df1.sort_values("award_datetime",ascending=False).drop_duplicates("songs")
                num_songs_awards.append(len(artist_df1["songs"].value_counts()))
                platinum_count.append(artist_df1[artist_df1["award"] == "Platinum"].sum()["award_num"])
                last_award_list.append(artist_df1.iloc[0]["award"])
                award_num_list.append(gold_count[-1] + platinum_count[-1])
            else:
                last_award_list.append(None)
                award_num_list.append(0)
                gold_count.append(0)
                platinum_count.append(0)
                num_songs_awards.append(0)
 
        max_index = award_num_list.index(max(award_num_list))
        song_df.loc[song,"last_award_type"] = last_award_list[max_index]
        song_df.loc[song,"award_num"] = award_num_list[max_index]
        song_df.loc[song,"gold_count"] = gold_count[max_index]
        song_df.loc[song,"platinum_count"] = platinum_count[max_index]
        song_df.loc[song,"num_songs_awards"] = num_songs_awards[max_index]
song_df["artist_has_award"] = song_df["award_num"] > 0

In [ ]:
song_df.groupby("album_label").count()["artist"].sort_values(ascending=False)

In [100]:
labels_occurence = list()
for i in np.asarray(song_df["album_label"]):
    if type(i) == str:
        label = i.split("/")
        for lab in label: 
            labels_occurence.append(lab.strip())

In [104]:
count_labels = list()
labels = set(labels_occurence)
for i in list(labels):
    count_labels.append([i,labels_occurence.count(i)])
label_df = pd.DataFrame(columns=["label","appearance_count"],data=count_labels)
label_df.sort_values("appearance_count",ascending=False)["appearance_count"][0:25].sum()

3936

In [105]:
label_df = label_df.sort_values("appearance_count",ascending=False)
labels = label_df["label"]
for label in label_df.index:
    label_stripped = label_df.loc[label,"label"]
    for i in labels:
        stopwords = ["Entertainment","Records","Recordings","LLC","Ltd."]
        i_stripped = i
        for word in stopwords:
            i_stripped = i_stripped.replace(word,"")
            label_stripped = label_stripped.replace(word,"")
        if  fuzz.partial_ratio(label_stripped,i_stripped) >= 90 and fuzz.ratio(label_stripped,i_stripped) >=65:
            if fuzz.ratio(label_df.loc[label,"label"],i) != 100:
                print(label_stripped,i_stripped,fuzz.ratio(label_df.loc[label,"label"],i))
            label_df.loc[label,"label_idx"] = i
            break

Atlantic  Atlantic 67
RCA  Label Nashville RCA  Label 77
Cash Money Cash Money  71
Universal Rec. Universal  84
Interscope  Interscope 71
Def Jam Def Jam  56
Universal Music Universal  69
Universal-Island   Universal  74
Republic Republic  67
Capitol  Nashville Capitol Nashville 81
Motown  Motown 60
Atlantic  UK Atlantic 59
Universal Republic  Universal  79
Sony Music  Sony Music Latin 70
Interscope * Interscope 69
Quality Control Music,  Quality Control Music 89
Nashville Arista Nashville 72
Sony Music UK Sony Music  59
Capitol Capitol  64
Grand Hustle,  Grand Hustle 83
Getting Out Our Dreams, Inc. Getting Out Our Dreams Inc. (G.O.O.D.) Music 75
Virgin EMI Virgin  58
Warner Bros. Warner  69
Geffen* Geffen 92
Rhino Atlantic Atlantic 73
EMI  Nashville Nashville 60
Young Money Young Money Ent. 81
Elektra Elektra  64
Syco Music UK Syco Music 87
Epic  Epic 50
2012 Ameritz Music  Ameritz Music Ltd 85
Cash Money  Inc. Cash Money  88
Capitol  (CAP) Capitol  83
Getting Out Our Dreams II Gettin

In [106]:
def label_mega_category(row):
    if row["appearance_count"] >= 200:
        row["label_category"] = "Mega Label"
    elif row["appearance_count"] >= 100:
        row["label_category"] = "Great Label"
    elif row["appearance_count"] >= 50:
        row["label_category"] = "Big Label"
    elif row["appearance_count"] >= 20:
        row["label_category"] = "Decent Label"
    elif row["appearance_count"] >= 10:
        row["label_category"] = "Small Label"
    else:
        row["label_category"] = "Mini Label"
    return row




        
label_df = label_df.apply(lambda row: label_mega_category(row),axis=1).sort_values("appearance_count",ascending=False)

In [107]:
label_df2 = label_df.groupby("label_idx").sum().sort_values("appearance_count",ascending=False)

def label_mega_category_groups(row):
    if row["appearance_count"] >= 200:
        row["label_category"] = "Mega Label"
    elif row["appearance_count"] >= 100:
        row["label_category"] = "Great Label"
    elif row["appearance_count"] >= 50:
        row["label_category"] = "Big Label"
    elif row["appearance_count"] >= 20:
        row["label_category"] = "Decent Label"
    elif row["appearance_count"] >= 10:
        row["label_category"] = "Small Label"
    else:
        row["label_category"] = "Mini Label"
    return row
label_df2 = label_df2.apply(lambda row: label_mega_category(row),axis=1).sort_values("appearance_count",ascending=False)

In [118]:
label_df = label_df.join(label_df2,on="label_idx",rsuffix="_idx")

In [119]:
label_df

,label,appearance_count,label_idx,label_category,appearance_count_idx,label_category_idx
651,Columbia,596,Columbia,Mega Label,599,Mega Label
595,Epic,351,Epic,Mega Label,366,Mega Label
973,Warner Records,268,Warner Records,Mega Label,288,Mega Label
58,RCA Records Label,223,RCA Records Label,Mega Label,317,Mega Label
550,Atlantic,211,Atlantic,Mega Label,467,Mega Label
...,...,...,...,...,...,...
261,4th & Broadway,1,4th & Broadway,Mini Label,1,Mini Label
262,OG Maco,1,OG Maco,Mini Label,1,Mini Label
267,Vertigo Berlin,1,Vertigo Berlin,Mini Label,1,Mini Label
200,101 Music,1,101 Music,Mini Label,1,Mini Label


In [129]:
for song in song_df.index:
    appearance_count = list()
    appearance_count_idx = list()
    label_category = list()
    label_category_idx = list()
    label = song_df.loc[song,"album_label"]
    if type(label) == str:
        label = label.split("/")
        for i in label:
            new_df = label_df[label_df["label"] == i.strip()]
            appearance_count_idx.append(new_df.iloc[0]["appearance_count_idx"])
            label_category_idx.append(new_df.iloc[0]["label_category_idx"])
            label_category.append(new_df.iloc[0]["label_category"])
            appearance_count.append(new_df.iloc[0]["appearance_count"])
        song_df.loc[song,"label_appearance_count"] = max(appearance_count)
        song_df.loc[song,"label_appearance_count_idx"] = max(appearance_count_idx)
        song_df.loc[song,"label_category"] = label_category[appearance_count.index(max(appearance_count))]
        song_df.loc[song,"label_category_idx"] = label_category_idx[appearance_count_idx.index(max(appearance_count_idx))]

['Mini Label'] [1]
['Great Label'] [107]
['Big Label'] [76]
['Mini Label', 'Mega Label'] [3, 211]
['Big Label'] [70]
['Mini Label', 'Mega Label'] [3, 211]
['Mini Label', 'Great Label'] [3, 108]
['Mega Label'] [596]
['Mini Label', 'Big Label'] [1, 70]
['Mini Label'] [4]
['Decent Label', 'Small Label'] [46, 16]
['Small Label', 'Big Label'] [16, 70]
['Great Label'] [107]
['Small Label', 'Mega Label'] [19, 351]
['Decent Label'] [41]
['Mini Label'] [1]
['Mini Label', 'Big Label'] [7, 70]
['Mega Label'] [596]
['Mini Label'] [3]
['Decent Label'] [41]
['Great Label'] [107]
['Big Label', 'Mega Label'] [51, 268]
['Small Label'] [10]
['Small Label', 'Great Label'] [13, 108]
['Mini Label'] [3]
['Mini Label', 'Big Label'] [3, 70]
['Decent Label'] [25]
['Big Label'] [94]
['Mini Label', 'Big Label'] [9, 70]
['Mega Label'] [268]
['Decent Label', 'Mini Label', 'Decent Label', 'Big Label'] [37, 9, 41, 91]
['Great Label'] [107]
['Mini Label', 'Decent Label'] [9, 49]
['Mini Label'] [1]
['Decent Label'] [4

['Mini Label', 'Mini Label'] [5, 5]
['Great Label'] [107]
['Mini Label'] [5]
['Small Label', 'Big Label'] [16, 70]
['Decent Label', 'Great Label'] [33, 108]
['Mini Label'] [9]
['Mini Label', 'Mini Label'] [2, 3]
['Mini Label'] [2]
['Decent Label'] [39]
['Mini Label', 'Big Label'] [7, 70]
['Great Label'] [194]
['Mini Label', 'Decent Label'] [9, 49]
['Decent Label', 'Mini Label', 'Decent Label', 'Big Label'] [37, 9, 41, 91]
['Small Label', 'Big Label'] [16, 70]
['Mega Label', 'Mini Label'] [351, 9]
['Small Label'] [10]
['Mega Label', 'Decent Label'] [351, 39]
['Mini Label', 'Mega Label'] [1, 268]
['Mini Label'] [1]
['Mega Label', 'Decent Label'] [351, 39]
['Mega Label', 'Decent Label'] [351, 39]
['Mini Label'] [1]
['Mini Label'] [1]
['Mini Label'] [7]
['Mini Label'] [1]
['Small Label'] [11]
['Big Label'] [50]
['Small Label', 'Small Label'] [14, 15]
['Mini Label', 'Small Label'] [2, 13]
['Mega Label', 'Small Label'] [351, 19]
['Small Label', 'Mega Label'] [14, 596]
['Decent Label'] [37]
[

['Big Label', 'Mega Label'] [51, 211]
['Mega Label', 'Small Label'] [211, 15]
['Big Label', 'Mega Label'] [51, 211]
['Decent Label', 'Decent Label', 'Big Label'] [37, 41, 91]
['Decent Label', 'Decent Label', 'Big Label'] [37, 41, 91]
['Big Label', 'Mega Label'] [51, 211]
['Big Label', 'Mega Label'] [51, 211]
['Big Label', 'Mega Label'] [51, 211]
['Big Label', 'Mega Label'] [51, 211]
['Big Label', 'Mega Label'] [51, 211]
['Great Label'] [107]
['Decent Label', 'Decent Label', 'Big Label'] [37, 41, 91]
['Big Label', 'Mega Label'] [51, 211]
['Mini Label'] [8]
['Decent Label', 'Decent Label', 'Big Label'] [37, 41, 91]
['Decent Label', 'Decent Label', 'Big Label'] [37, 41, 91]
['Big Label', 'Decent Label'] [85, 25]
['Big Label'] [94]
['Decent Label'] [29]
['Big Label', 'Decent Label'] [85, 25]
['Mini Label', 'Mini Label'] [3, 3]
['Small Label'] [11]
['Small Label'] [16]
['Great Label'] [107]
['Small Label'] [11]
['Small Label'] [19]
['Big Label'] [94]
['Decent Label', 'Mini Label', 'Decent L

['Decent Label'] [29]
['Mini Label'] [3]
['Great Label'] [178]
['Mini Label', 'Decent Label'] [4, 40]
['Mini Label'] [8]
['Mini Label'] [8]
['Mini Label'] [7]
['Mini Label'] [5]
['Mini Label'] [8]
['Small Label'] [15]
['Decent Label'] [29]
['Small Label'] [16]
['Big Label'] [50]
['Mini Label'] [6]
['Decent Label', 'Decent Label'] [32, 36]
['Great Label'] [178]
['Mini Label'] [1]
['Mega Label'] [268]
['Mini Label', 'Mini Label'] [8, 8]
['Decent Label', 'Decent Label'] [32, 36]
['Mega Label'] [596]
['Mini Label', 'Mini Label'] [1, 1]
['Decent Label', 'Great Label'] [21, 178]
['Decent Label', 'Decent Label'] [32, 36]
['Mini Label', 'Mini Label'] [1, 1]
['Decent Label', 'Decent Label'] [32, 36]
['Decent Label', 'Great Label'] [21, 178]
['Decent Label', 'Great Label'] [21, 178]
['Decent Label', 'Great Label'] [21, 178]
['Mini Label'] [7]
['Decent Label', 'Great Label'] [21, 178]
['Decent Label', 'Great Label'] [21, 178]
['Decent Label', 'Great Label'] [21, 178]
['Decent Label', 'Great Label

['Mega Label'] [268]
['Mini Label'] [1]
['Great Label'] [121]
['Decent Label'] [39]
['Mini Label'] [1]
['Great Label'] [194]
['Big Label'] [56]
['Mini Label'] [2]
['Mega Label'] [223]
['Great Label'] [114]
['Big Label'] [94]
['Small Label', 'Mega Label'] [16, 211]
['Small Label', 'Mega Label'] [14, 596]
['Mini Label', 'Great Label'] [1, 108]
['Big Label'] [56]
['Mega Label'] [351]
['Big Label'] [56]
['Mini Label'] [6]
['Mini Label', 'Mini Label', 'Great Label'] [8, 8, 108]
['Mega Label', 'Mini Label', 'Mini Label'] [351, 6, 7]
['Mega Label', 'Small Label'] [351, 19]
['Big Label'] [98]
['Mega Label'] [351]
['Decent Label'] [25]
['Mega Label'] [351]
['Mega Label'] [351]
['Small Label', 'Great Label'] [13, 108]
['Big Label'] [50]
['Great Label'] [107]
['Small Label', 'Decent Label'] [11, 49]
['Big Label'] [98]
['Mega Label'] [223]
['Mini Label', 'Great Label'] [6, 108]
['Mega Label', 'Mini Label', 'Mini Label'] [351, 6, 7]
['Mega Label', 'Mini Label', 'Mini Label'] [351, 6, 7]
['Decent La

...

['Small Label'] [13]
['Small Label'] [10]
['Great Label'] [121]
['Big Label'] [73]
['Mega Label'] [351]
['Decent Label'] [48]
['Big Label'] [57]
['Small Label', 'Great Label'] [14, 121]
['Small Label'] [12]
['Big Label'] [73]
['Big Label'] [98]
['Great Label'] [153]
['Mega Label'] [223]
['Small Label'] [11]
['Mega Label', 'Decent Label'] [268, 29]
['Big Label'] [91]
['Decent Label'] [23]
['Mini Label'] [4]
['Great Label'] [153]
['Great Label'] [194]
['Small Label'] [19]
['Mini Label'] [5]
['Decent Label'] [22]
['Small Label'] [10]
['Big Label'] [85]
['Mini Label'] [1]
['Mini Label'] [1]
['Decent Label'] [23]
['Great Label'] [153]
['Big Label'] [58]
['Small Label'] [13]
['Mega Label'] [596]
['Decent Label', 'Big Label'] [20, 73]
['Great Label'] [153]
['Mini Label'] [1]
['Decent Label'] [23]
['Big Label'] [60]
['Small Label'] [14]
['Mini Label'] [4]
['Mega Label'] [596]
['Mega Label'] [351]
['Decent Label'] [21]
['Great Label', 'Big Label'] [153, 73]
['Decent Label'] [37]
['Decent Label'

['Decent Label'] [20]
['Mini Label'] [5]
['Mega Label'] [596]
['Great Label'] [153]
['Mini Label'] [3]
['Decent Label'] [35]
['Great Label'] [183]
['Mini Label'] [2]
['Big Label'] [84]
['Mini Label', 'Mega Label'] [5, 351]
['Big Label'] [85]
['Great Label'] [153]
['Big Label'] [76]
['Mini Label', 'Mini Label'] [1, 2]
['Mega Label', 'Mini Label'] [351, 7]
['Great Label'] [121]
['Great Label'] [183]
['Decent Label'] [36]
['Mega Label'] [268]
['Big Label'] [69]
['Big Label'] [94]
['Small Label'] [17]
['Mini Label'] [1]
['Mini Label'] [8]
['Small Label'] [12]
['Mega Label'] [351]
['Small Label'] [10]
['Mega Label', 'Mini Label'] [211, 9]
['Small Label'] [18]
['Mega Label', 'Big Label'] [596, 73]
['Mini Label'] [5]
['Small Label'] [11]
['Mini Label', 'Small Label', 'Mega Label'] [3, 15, 596]
['Big Label'] [73]
['Decent Label'] [31]
['Mini Label'] [8]
['Mega Label'] [268]
['Small Label'] [12]
['Mini Label'] [4]
['Great Label'] [121]
['Big Label'] [85]
['Decent Label'] [36]
['Decent Label'] [

['Great Label'] [153]
['Great Label'] [121]
['Great Label'] [178]
['Great Label'] [121]
['Mega Label'] [268]
['Decent Label'] [37]
['Mini Label'] [2]
['Big Label'] [85]
['Big Label'] [76]
['Small Label'] [18]
['Great Label'] [121]
['Big Label'] [98]
['Mega Label', 'Decent Label'] [351, 29]
['Mini Label'] [5]
['Decent Label'] [28]
['Mini Label'] [1]
['Mega Label'] [268]
['Mini Label'] [2]
['Big Label'] [85]
['Mega Label'] [351]
['Mini Label'] [8]
['Great Label'] [114]
['Decent Label', 'Small Label'] [35, 16]
['Mega Label', 'Mini Label'] [211, 9]
['Great Label'] [183]
['Great Label'] [121]
['Decent Label'] [23]
['Mini Label'] [1]
['Mini Label'] [2]
['Big Label'] [85]
['Decent Label', 'Big Label'] [20, 73]
['Small Label'] [12]
['Mega Label', 'Mini Label'] [351, 6]
['Mega Label'] [268]
['Decent Label'] [36]
['Big Label'] [60]
['Mega Label'] [223]
['Small Label'] [17]
['Decent Label'] [27]
['Mini Label'] [4]
['Small Label'] [11]
['Small Label'] [19]
['Small Label'] [10]
['Great Label'] [103

['Mega Label'] [268]
['Great Label'] [121]
['Mini Label'] [7]
['Mega Label'] [351]
['Decent Label', 'Big Label'] [20, 73]
['Big Label'] [69]
['Mini Label', 'Mini Label'] [1, 3]
['Mega Label'] [596]
['Great Label'] [178]
['Great Label'] [183]
['Mega Label'] [223]
['Mini Label'] [2]
['Big Label'] [77]
['Mini Label'] [5]
['Decent Label'] [22]
['Decent Label'] [31]
['Mega Label', 'Big Label'] [596, 73]
['Big Label'] [69]
['Mini Label'] [1]
['Mega Label'] [596]
['Mega Label'] [596]
['Great Label'] [183]
['Mega Label'] [351]
['Small Label'] [10]
['Mega Label'] [596]
['Mega Label', 'Decent Label'] [351, 29]
['Great Label'] [194]
['Great Label'] [194]
['Big Label'] [58]
['Big Label'] [84]
['Mini Label'] [1]
['Small Label'] [14]
['Decent Label'] [47]
['Mini Label'] [4]
['Mini Label'] [1]
['Mega Label'] [223]
['Mega Label'] [268]
['Great Label'] [103]
['Mega Label'] [596]
['Decent Label'] [23]
['Mini Label'] [1]
['Great Label'] [153]
['Mini Label'] [1]
['Mini Label', 'Mega Label'] [4, 211]
['Dec

['Big Label'] [77]
['Mega Label'] [223]
['Mega Label', 'Decent Label'] [268, 29]
['Great Label'] [114]
['Great Label'] [121]
['Big Label'] [69]
['Decent Label'] [22]
['Small Label'] [19]
['Big Label'] [94]
['Big Label'] [98]
['Big Label'] [98]
['Great Label'] [183]
['Big Label'] [69]
['Mega Label'] [596]
['Big Label'] [85]
['Great Label'] [153]
['Mega Label'] [596]
['Mini Label', 'Mega Label'] [5, 596]
['Mini Label'] [1]
['Mini Label'] [1]
['Decent Label', 'Small Label'] [35, 16]
['Great Label'] [121]
['Decent Label', 'Big Label'] [20, 73]
['Decent Label'] [37]
['Big Label'] [91]
['Mini Label'] [7]
['Small Label', 'Mega Label'] [14, 268]
['Mini Label'] [5]
['Great Label'] [183]
['Great Label', 'Big Label'] [121, 57]
['Small Label'] [17]
['Mega Label', 'Decent Label'] [351, 29]
['Mini Label'] [6]
['Decent Label'] [27]
['Mini Label'] [2]
['Great Label'] [153]
['Mini Label'] [1]
['Mini Label'] [8]
['Decent Label', 'Mega Label'] [35, 268]
['Big Label'] [77]
['Big Label', 'Small Label'] [58

In [130]:
song_df.pop()

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,firstrank,top10,top50,top25,top75,top5,label_appearance_count,appearance_count_idx,label_category,label_category_idx
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,85,True,True,True,True,True,1.0,1.0,Mini Label,Mini Label
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,7,True,True,True,True,True,107.0,161.0,Great Label,Great Label
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,0TK2YIli7K1leLovkQiNik,False,190960.0,1.0,1.0,...,2,True,True,True,True,True,76.0,92.0,Big Label,Big Label
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,50,True,True,True,True,True,211.0,467.0,Mega Label,Mega Label
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,15,True,True,True,True,True,70.0,297.0,Big Label,Mega Label
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8514,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,79,False,False,False,False,False,60.0,92.0,Big Label,Big Label
8515,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,93,False,False,False,False,False,69.0,69.0,Big Label,Big Label
8516,Sting & The Police,Roxanne `97 - Puff Daddy Remix,False,95,1998-03-21,0RlWNvJKKp0gODH7sUwm1j,False,275493.0,1.0,7.0,...,95,False,False,False,False,False,2.0,58.0,Mini Label,Big Label
8517,Shania Twain,Don't Be Stupid (You Know I Love You),False,99,1998-03-21,6xAF3xYDK915NEvC6WNbj9,False,214240.0,1.0,4.0,...,99,False,False,False,False,False,73.0,73.0,Big Label,Big Label


In [ ]:
rec_df["join"] = rec_df["artist"] + rec_df["title"]
song_df["join"] = song_df["artist"] + song_df["title"]
rec_df = rec_df.sort_values("date",ascending=False).drop_duplicates(["artist","title"])
rec_df.pop("artist")
rec_df.pop("isnew")
rec_df.pop("rank")
rec_df.pop("date")
rec_df.pop("title")
song_df = song_df.merge(rec_df,on="join")

In [ ]:
song_df["top10"] = song_df["bestrank"] < 10
song_df["top50"] = song_df["bestrank"] < 50
song_df["top25"] = song_df["bestrank"] < 25
song_df["top75"] = song_df["bestrank"] < 75
song_df["top5"] = song_df["bestrank"] < 5

In [ ]:
with open('song_df_aggregate.pkl', 'wb') as f:
    pickle.dump(song_df, f)